In [ ]:
# ------------------------------------------------------------------
# Example: memory-efficient statistics on a large NetCDF
# ------------------------------------------------------------------
# What “chunks” means:
#   Xarray can read the file in small blocks (chunks) instead of all
#   at once.  Only the blocks you actually need are pulled into RAM,
#   so even a laptop can handle a 30-m grid for all of Ontario. Read
#   more at: docs.xarray.dev/en/stable/generated/xarray.DataArray.chunk.html
# ------------------------------------------------------------------

In [ ]:
#Import xarray (handles NetCDF)
import xarray as xr
import dask.array as da

In [ ]:
#Open the NetCDF file in “chunks” so only small pieces load at a time.
#Then pick our variable and convert to float32 to cut memory use in half.
#    (file must be saved in ../Data/)
ds = xr.open_dataset(
    "../Data/Ontario_30m_Manure_FW_N_P_kg_ha.nc",
    chunks="auto"        # This tells xarray to read in small chunks on demand,
                         # so it never pulls the entire array into RAM at once.
)

#Choose the variable we care about: total manure (kg/ha/yr)
var = ds["total_manure"].chunk({"lat_5070": 2048, "lon_5070": 2048})
var = var.astype("float32")          # halve memory per chunk

In [ ]:
#Replace zeros with NaN but keep it as a Dask array for later
nz = var.where(var > 0).data         

In [ ]:
#Quickly find the smallest and largest non-zero values.
#These two numbers define the range we’ll histogram over.
min_nz = da.nanmin(nz).compute()
max_nz = da.nanmax(nz).compute()

In [ ]:
#If the data had no values > 0, stop here.
if np.isnan(min_nz) or np.isnan(max_nz):
    print("No non-zero cells found in this variable.")
    quit()

In [ ]:
#Build a simple histogram with 1 000 bins between min_nz and max_nz.
#Dask fills counts chunk by chunk; only 1 000 counts + edges land in RAM.
hist, edges = da.histogram(nz, bins=1000, range=(min_nz, max_nz))
hist, edges = da.compute(hist, edges)

In [ ]:
#Turn those counts into a running total so we can read off quartiles.
cum = np.cumsum(hist)
total = cum[-1]

In [ ]:
def percentile_from_hist(p):
    """Find the value at percentile p (0–1) by interpolating within histogram bins."""
    rank = p * total
    idx = np.searchsorted(cum, rank)
    if idx == 0:
        return edges[0]
    # How far into this bin is our rank? Interpolate linearly.
    prev = cum[idx - 1]
    frac = (rank - prev) / hist[idx]
    return edges[idx] + frac * (edges[idx + 1] - edges[idx])

q1     = percentile_from_hist(0.25)   # 25th percentile
median = percentile_from_hist(0.50)   # 50th percentile
q3     = percentile_from_hist(0.75)   # 75th percentile

In [ ]:
#Print the exact values without rounding
print("Total Manure – non-zero cells only")
print(f"  Q1  (25th pct) : {q1}")
print(f"  Median         : {median}")
print(f"  Q3  (75th pct) : {q3}")
print(f"  Min (non-zero) : {min_nz}")
print(f"  Max            : {max_nz}")